In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
try:
    spark.stop()
except NameError:
    pass  # Spark not defined yet, safe to ignore

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("EcommerceETL") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/20 22:13:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_sales_raw = spark.read.csv("../ecommerce_sales.csv", header=True)

In [5]:
from pyspark.sql.functions import col, count, when

df_sales_raw.select([count(when(col(c).isNull(), c)).alias(c) for c in df_sales_raw.columns]).show()

+-----+--------+----+------+----------+--------------+------------------+-----+---+--------+----+----+--------------+---+--------+------+---------+----------+----------------+------------+-------------+---+------------+-----------+
|index|Order ID|Date|Status|Fulfilment|Sales Channel |ship-service-level|Style|SKU|Category|Size|ASIN|Courier Status|Qty|currency|Amount|ship-city|ship-state|ship-postal-code|ship-country|promotion-ids|B2B|fulfilled-by|Unnamed: 22|
+-----+--------+----+------+----------+--------------+------------------+-----+---+--------+----+----+--------------+---+--------+------+---------+----------+----------------+------------+-------------+---+------------+-----------+
|    0|       0|   0|     0|         0|             0|                 0|    0|  0|       0|   0|   0|          6872|  0|    7795|  7795|       33|        33|              33|          33|        49153|  0|       89698|      49050|
+-----+--------+----+------+----------+--------------+------------------

In [6]:
from pyspark.sql.functions import col, to_date, when, regexp_replace, expr

df_sales = df_sales_raw.select(
    to_date(col("Date"), "MM-dd-yy").alias("Date"),
    col("Status"),
    col("Fulfilment"),
    when(col("Style").isNull() | (col("Style") == ""), "Unknown").otherwise(col("Style")).alias("Style"),
    when(col("SKU").isNull() | (col("SKU") == ""), "Unknown").otherwise(col("SKU")).alias("SKU"),
    col("ASIN"),
    when(col("Courier Status").isNull() | (col("Courier Status") == ""), "Unknown").otherwise(col("Style")).alias("Courier_Status"),
    col("Qty").cast("float").cast("int"),
    col("Amount").cast("float"),
    col("B2B").cast("boolean"),
    col("currency").alias("Currency")
).filter(col("Amount").isNotNull() & col("Qty").isNotNull())

In [7]:
df_sales.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier_Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- Amount: float (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- Currency: string (nullable = true)



In [8]:
df_sales.show(5)

+----------+--------------------+----------+-------+---------------+----------+--------------+---+------+-----+--------+
|      Date|              Status|Fulfilment|  Style|            SKU|      ASIN|Courier_Status|Qty|Amount|  B2B|Currency|
+----------+--------------------+----------+-------+---------------+----------+--------------+---+------+-----+--------+
|2022-04-30|           Cancelled|  Merchant| SET389| SET389-KR-NP-S|B09KXVBD7Z|       Unknown|  0|647.62|false|     INR|
|2022-04-30|Shipped - Deliver...|  Merchant|JNE3781|JNE3781-KR-XXXL|B09K3WFS32|       JNE3781|  1| 406.0|false|     INR|
|2022-04-30|             Shipped|    Amazon|JNE3371|  JNE3371-KR-XL|B07WV4JV4D|       JNE3371|  1| 329.0| true|     INR|
|2022-04-30|           Cancelled|  Merchant|  J0341|     J0341-DR-L|B099NRCT7B|       Unknown|  0|753.33|false|     INR|
|2022-04-30|             Shipped|    Amazon|JNE3671|JNE3671-TU-XXXL|B098714BZP|       JNE3671|  1| 574.0|false|     INR|
+----------+--------------------

In [9]:
df_sales.select([count(when(col(c).isNull(), c)).alias(c) for c in df_sales.columns]).show()

+----+------+----------+-----+---+----+--------------+---+------+---+--------+
|Date|Status|Fulfilment|Style|SKU|ASIN|Courier_Status|Qty|Amount|B2B|Currency|
+----+------+----------+-----+---+----+--------------+---+------+---+--------+
|   0|     0|         0|    0|  0|   0|             0|  0|     0|  0|       0|
+----+------+----------+-----+---+----+--------------+---+------+---+--------+



In [10]:
df_international_raw = spark.read.csv("../international_sales.csv", header=True)

In [11]:
df_international_raw.select([count(when(col(c).isNull(), c)).alias(c) for c in df_international_raw.columns]).show()

+-----+----+------+--------+-----+----+----+----+----+---------+
|index|DATE|Months|CUSTOMER|Style| SKU|Size| PCS|RATE|GROSS AMT|
+-----+----+------+--------+-----+----+----+----+----+---------+
|    0|   1|    25|    1040| 1040|2474|1040|1040|1040|     1040|
+-----+----+------+--------+-----+----+----+----+----+---------+



In [12]:
df_cleaned_raw = df_international_raw.withColumn(
    "GROSS_AMT_CLEAN", regexp_replace(col("GROSS AMT"), ",", "")
)

df_valid_dates = df_cleaned_raw.filter(col("DATE").rlike("^[0-9]{2}-[0-9]{2}-[0-9]{2}$"))

df_international = df_valid_dates.select(
    to_date(col("DATE"), "MM-dd-yy").alias("Date"), 
    col("Months").alias("Month"),
    col("CUSTOMER"),
    when(col("Style").isNull() | (col("Style") == ""), "Unknown").otherwise(col("Style")).alias("Style"),
    when(col("SKU").isNull() | (col("SKU") == ""), "Unknown").otherwise(col("SKU")).alias("SKU"),
    when(col("Size").isNull() | (col("Size") == ""), "Unknown").otherwise(col("Size")).alias("Size"),
    col("PCS").cast("float").cast("int").alias("Pieces"),
    col("RATE").cast("float").alias("Rate"),
    expr("try_cast(GROSS_AMT_CLEAN as float)").alias("Gross_Amount")
)

df_international = df_international.filter(col("Gross_Amount").isNotNull())

In [13]:
df_international.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Month: string (nullable = true)
 |-- CUSTOMER: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Pieces: integer (nullable = true)
 |-- Rate: float (nullable = true)
 |-- Gross_Amount: float (nullable = true)



In [14]:
df_international.select([count(when(col(c).isNull(), c)).alias(c) for c in df_international.columns]).show()

+----+-----+--------+-----+---+----+------+----+------------+
|Date|Month|CUSTOMER|Style|SKU|Size|Pieces|Rate|Gross_Amount|
+----+-----+--------+-----+---+----+------+----+------------+
|   0|    0|       0|    0|  0|   0|     0|   0|           0|
+----+-----+--------+-----+---+----+------+----+------------+



In [15]:
df_pricing_raw = spark.read.csv("../may-2022.csv", header=True)

In [16]:
df_pricing_raw.select([count(when(col(c).isNull(), c)).alias(c) for c in df_pricing_raw.columns]).show()
df_pricing_raw.printSchema()

+-----+---+--------+-------+--------+------+---+-------+-------------+--------+----------+--------------+------------+------------+----------+---------+------------+
|index|Sku|Style Id|Catalog|Category|Weight| TP|MRP Old|Final MRP Old|Ajio MRP|Amazon MRP|Amazon FBA MRP|Flipkart MRP|Limeroad MRP|Myntra MRP|Paytm MRP|Snapdeal MRP|
+-----+---+--------+-------+--------+------+---+-------+-------------+--------+----------+--------------+------------+------------+----------+---------+------------+
|    0|  0|       0|      0|       0|     0|  0|      0|            0|       0|         0|             0|           0|           0|         0|        0|           0|
+-----+---+--------+-------+--------+------+---+-------+-------------+--------+----------+--------------+------------+------------+----------+---------+------------+

root
 |-- index: string (nullable = true)
 |-- Sku: string (nullable = true)
 |-- Style Id: string (nullable = true)
 |-- Catalog: string (nullable = true)
 |-- Category

In [17]:
int_columns = [
    "Weight", "TP", "MRP Old", "Final MRP Old", "Ajio MRP", "Amazon MRP", "Amazon FBA MRP",
    "Flipkart MRP", "Limeroad MRP", "Myntra MRP", "Paytm MRP", "Snapdeal MRP"
]

df_pricing = df_pricing_raw.select(
    col("index"),
    col("Sku"),
    col("Catalog"),
    col("Category"),
    *[
        when(
            regexp_replace(col(c), ",", "").rlike("^[0-9]+$"),
            regexp_replace(col(c), ",", "").cast("int")
        ).otherwise(None).alias(c) if c in int_columns else col(c)
        for c in df_pricing_raw.columns if c not in ["index", "Sku", "Catalog", "Category"]
    ]
)

In [18]:
df_pricing.printSchema()

root
 |-- index: string (nullable = true)
 |-- Sku: string (nullable = true)
 |-- Catalog: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Style Id: string (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- TP: integer (nullable = true)
 |-- MRP Old: integer (nullable = true)
 |-- Final MRP Old: integer (nullable = true)
 |-- Ajio MRP: integer (nullable = true)
 |-- Amazon MRP: integer (nullable = true)
 |-- Amazon FBA MRP: integer (nullable = true)
 |-- Flipkart MRP: integer (nullable = true)
 |-- Limeroad MRP: integer (nullable = true)
 |-- Myntra MRP: integer (nullable = true)
 |-- Paytm MRP: integer (nullable = true)
 |-- Snapdeal MRP: integer (nullable = true)



In [55]:
from pyspark.sql.functions import date_format
df_sales = df_sales.withColumn("Month", date_format("Date", "yyyy-MM"))

In [56]:
df_sales.createOrReplaceTempView("sales")
df_international.createOrReplaceTempView("international_sales")
df_pricing.createOrReplaceTempView("pricing")

In [57]:
print(df_sales.columns)

['Date', 'Status', 'Fulfilment', 'Style', 'SKU', 'ASIN', 'Courier_Status', 'Qty', 'Amount', 'B2B', 'Currency', 'Month']


In [63]:
df_sales.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier_Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- Amount: float (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- Currency: string (nullable = true)
 |-- Month: string (nullable = true)



In [64]:
print(df_international.columns)

['Date', 'Month', 'CUSTOMER', 'Style', 'SKU', 'Size', 'Pieces', 'Rate', 'Gross_Amount']


In [65]:
print(df_pricing.columns)

['index', 'Sku', 'Catalog', 'Category', 'Style Id', 'Weight', 'TP', 'MRP Old', 'Final MRP Old', 'Ajio MRP', 'Amazon MRP', 'Amazon FBA MRP', 'Flipkart MRP', 'Limeroad MRP', 'Myntra MRP', 'Paytm MRP', 'Snapdeal MRP']


In [66]:
spark.sql("""
CREATE OR REPLACE TEMP VIEW top_categories AS
SELECT Style AS Category
FROM sales
GROUP BY Style
ORDER BY SUM(Amount) DESC
LIMIT 5
""")

monthly_sales_by_category = spark.sql("""
SELECT 
  Month, 
  Style AS Category, 
  SUM(Amount) AS Revenue
FROM sales
WHERE Style IN (SELECT Category FROM top_categories)
  AND Month IS NOT NULL
GROUP BY Month, Style
ORDER BY Month, Category
""")
# line chart
monthly_sales_by_category = monthly_sales_by_category.withColumn("Month", to_date("Month", "yyyy-MM"))

In [67]:
monthly_sales_by_category.printSchema()

root
 |-- Month: date (nullable = true)
 |-- Category: string (nullable = true)
 |-- Revenue: double (nullable = true)



In [68]:
monthly_sales_by_category.show(10, truncate=False)
monthly_sales_by_category.write.mode("overwrite").parquet("outputs/monthly_sales_by_category")

+----------+--------+------------------+
|Month     |Category|Revenue           |
+----------+--------+------------------+
|2022-03-01|J0003   |3440.0            |
|2022-03-01|J0230   |2997.0            |
|2022-03-01|J0341   |2198.0            |
|2022-03-01|JNE3797 |2246.760009765625 |
|2022-03-01|SET268  |2094.0            |
|2022-04-01|J0003   |475522.47900390625|
|2022-04-01|J0230   |888995.1998901367 |
|2022-04-01|J0341   |489053.42028808594|
|2022-04-01|JNE3797 |319696.0498046875 |
|2022-04-01|SET268  |1060213.7199707031|
+----------+--------+------------------+
only showing top 10 rows


In [69]:
top_product_sku_by_amount = spark.sql("""
SELECT SKU, SUM(Amount) AS Total_Gross, COUNT(*) AS Orders
FROM sales
GROUP BY SKU
ORDER BY Total_Gross DESC
LIMIT 10    
""")
# bar chart

In [70]:
top_product_sku_by_amount.show(10, truncate=False)
top_product_sku_by_amount.write.mode("overwrite").parquet("outputs/top_product_sku_by_amount")

+---------------+------------------+------+
|SKU            |Total_Gross       |Orders|
+---------------+------------------+------+
|J0230-SKD-M    |527699.1999511719 |472   |
|JNE3797-KR-L   |524581.76953125   |722   |
|J0230-SKD-S    |479937.14013671875|428   |
|JNE3797-KR-M   |454290.15966796875|621   |
|JNE3797-KR-S   |407302.56951904297|553   |
|JNE3797-KR-XL  |332155.23974609375|446   |
|J0230-SKD-L    |305616.9500732422 |275   |
|JNE3797-KR-XS  |303616.69982910156|407   |
|SET268-KR-NP-XL|284058.9599609375 |372   |
|JNE3797-KR-XXXL|276375.7998046875 |374   |
+---------------+------------------+------+



In [71]:
sales_by_fulfilment = spark.sql("""
SELECT Fulfilment, SUM(Amount) AS Revenue, COUNT(*) AS Orders
FROM sales
GROUP BY Fulfilment
""")
# pie chart

In [72]:
sales_by_fulfilment.show(10, truncate=False)
sales_by_fulfilment.write.mode("overwrite").parquet("outputs/sales_by_fulfilment")

+----------+-------------------+------+
|Fulfilment|Revenue            |Orders|
+----------+-------------------+------+
|Merchant  |2.427052729008484E7|37541 |
|Amazon    |5.4322151E7        |83639 |
+----------+-------------------+------+



In [73]:
monthly_sales_by_customer = spark.sql("""
SELECT Month, CUSTOMER, SUM(Gross_Amount) AS Revenue
FROM international_sales
GROUP BY Month, CUSTOMER
ORDER BY Month
LIMIT 10
""")
# line chart

In [74]:
monthly_sales_by_customer.show(10, truncate=False)
monthly_sales_by_customer.write.mode("overwrite").parquet("outputs/monthly_sales_by_customer")

+------+-------------------------------+-------+
|Month |CUSTOMER                       |Revenue|
+------+-------------------------------+-------+
|Apr-22|Tanusree Majumder              |9562.5 |
|Apr-22|Natheliya                      |35675.0|
|Apr-22|Fusion Fashions Corp. (Gopikas)|67840.0|
|Apr-22|Rakshanda Mahajan              |25938.5|
|Apr-22|Dhenooga Ravintheran           |10861.5|
|Apr-22|Priyanka Gupta                 |53260.0|
|Apr-22|Ria Fashion                    |10090.0|
|Apr-22|Amandeep Kaur                  |21950.0|
|Apr-22|Avin                           |10987.5|
|Apr-22|Jeannie Thein Win              |71400.0|
+------+-------------------------------+-------+



In [75]:
top_intl_product_sku_by_amount = spark.sql("""
SELECT SKU, SUM(Gross_Amount) AS Total_Gross_Amount
FROM international_sales
WHERE SKU != 'Unknown'
GROUP BY SKU
ORDER BY Total_Gross_Amount DESC
LIMIT 10
""")
# bar chart

In [76]:
top_intl_product_sku_by_amount.show(10, truncate=False)
top_intl_product_sku_by_amount.write.mode("overwrite").parquet("outputs/top_intl_product_sku_by_amount")

+---------------+------------------+
|SKU            |Total_Gross_Amount|
+---------------+------------------+
|SET268-KR-NP-L |50618.0           |
|J0277-SKD-S    |42631.0           |
|J0277-SKD-M    |37633.0           |
|SET268-KR-NP-M |33866.0           |
|SET268-KR-NP-XL|29249.0           |
|SET268-KR-NP-S |25505.0           |
|SET252-KR-PP-S.|24911.0           |
|SET110-KR-PP-S |24443.0           |
|SET110-KR-PP-M |24428.0           |
|J0277-SKD-L    |23821.0           |
+---------------+------------------+



In [77]:
pricing_across_platforms = spark.sql("""
SELECT
  Sku,
  `Amazon MRP`,
  `Amazon FBA MRP`,
  `Myntra MRP`,
  `Ajio MRP`,
  `Flipkart MRP`,
  `Snapdeal MRP`,
  `Paytm MRP`,
  `Limeroad MRP`
FROM pricing
ORDER BY Sku DESC
""")

In [78]:
pricing_across_platforms.show(20, truncate=False)
pricing_across_platforms.write.mode("overwrite").parquet("outputs/pricing_across_platforms")

+----------------+----------+--------------+----------+--------+------------+------------+---------+------------+
|Sku             |Amazon MRP|Amazon FBA MRP|Myntra MRP|Ajio MRP|Flipkart MRP|Snapdeal MRP|Paytm MRP|Limeroad MRP|
+----------------+----------+--------------+----------+--------+------------+------------+---------+------------+
|TULIP - 1 BAG_L |498       |498           |498       |498     |498         |498         |498      |498         |
|SILVER3-1296-2XL|3149      |3149          |3149      |3149    |3149        |3149        |3149     |3149        |
|SILVER3-1293-L  |3149      |3149          |3149      |3149    |3149        |3149        |3149     |3149        |
|SILVER3-1293-2XL|3149      |3149          |3149      |3149    |3149        |3149        |3149     |3149        |
|Raag-1365_XL    |1365      |1365          |1365      |1365    |1365        |1365        |1365     |1365        |
|PLATINUM        |1695      |1695          |1695      |1695    |1695        |1695       

In [80]:
max_price_diff_across_platforms = spark.sql("""
SELECT
  SKU,
  `Amazon MRP`,
  `Amazon FBA MRP`,
  `Myntra MRP`,
  `Ajio MRP`,
  `Flipkart MRP`,
  `Snapdeal MRP`,
  `Paytm MRP`,
  `Limeroad MRP`,
  
  GREATEST(
    `Amazon MRP`, `Amazon FBA MRP`, `Myntra MRP`, `Ajio MRP`,
    `Flipkart MRP`, `Snapdeal MRP`, `Paytm MRP`, `Limeroad MRP`
  ) -
  LEAST(
    `Amazon MRP`, `Amazon FBA MRP`, `Myntra MRP`, `Ajio MRP`,
    `Flipkart MRP`, `Snapdeal MRP`, `Paytm MRP`, `Limeroad MRP`
  ) AS Max_Channel_Diff,

  CASE GREATEST(
    `Amazon MRP`, `Amazon FBA MRP`, `Myntra MRP`, `Ajio MRP`,
    `Flipkart MRP`, `Snapdeal MRP`, `Paytm MRP`, `Limeroad MRP`
  )
    WHEN `Amazon MRP` THEN 'Amazon MRP'
    WHEN `Amazon FBA MRP` THEN 'Amazon FBA MRP'
    WHEN `Myntra MRP` THEN 'Myntra MRP'
    WHEN `Ajio MRP` THEN 'Ajio MRP'
    WHEN `Flipkart MRP` THEN 'Flipkart MRP'
    WHEN `Snapdeal MRP` THEN 'Snapdeal MRP'
    WHEN `Paytm MRP` THEN 'Paytm MRP'
    WHEN `Limeroad MRP` THEN 'Limeroad MRP'
  END AS Max_Platform,

  CASE LEAST(
    `Amazon MRP`, `Amazon FBA MRP`, `Myntra MRP`, `Ajio MRP`,
    `Flipkart MRP`, `Snapdeal MRP`, `Paytm MRP`, `Limeroad MRP`
  )
    WHEN `Amazon MRP` THEN 'Amazon MRP'
    WHEN `Amazon FBA MRP` THEN 'Amazon FBA MRP'
    WHEN `Myntra MRP` THEN 'Myntra MRP'
    WHEN `Ajio MRP` THEN 'Ajio MRP'
    WHEN `Flipkart MRP` THEN 'Flipkart MRP'
    WHEN `Snapdeal MRP` THEN 'Snapdeal MRP'
    WHEN `Paytm MRP` THEN 'Paytm MRP'
    WHEN `Limeroad MRP` THEN 'Limeroad MRP'
  END AS Min_Platform

FROM pricing
ORDER BY Max_Channel_Diff DESC
""")

In [82]:
max_price_diff_across_platforms.show(truncate=False)
max_price_diff_across_platforms.write.mode("overwrite").parquet("outputs/max_price_diff_across_platforms")

+---------+----------+--------------+----------+--------+------------+------------+---------+------------+----------------+------------+------------+
|SKU      |Amazon MRP|Amazon FBA MRP|Myntra MRP|Ajio MRP|Flipkart MRP|Snapdeal MRP|Paytm MRP|Limeroad MRP|Max_Channel_Diff|Max_Platform|Min_Platform|
+---------+----------+--------------+----------+--------+------------+------------+---------+------------+----------------+------------+------------+
|Os168_S  |2495      |2495          |1695      |1695    |1695        |1695        |1695     |1695        |800             |Amazon MRP  |Myntra MRP  |
|Os168_M  |2495      |2495          |1695      |1695    |1695        |1695        |1695     |1695        |800             |Amazon MRP  |Myntra MRP  |
|Os168_L  |2495      |2495          |1695      |1695    |1695        |1695        |1695     |1695        |800             |Amazon MRP  |Myntra MRP  |
|Os168_XL |2495      |2495          |1695      |1695    |1695        |1695        |1695     |1695   

In [83]:
avg_price_per_sku = spark.sql("""
SELECT
  Sku,
  AVG(`Amazon MRP`) AS Avg_Amazon,
  AVG(`Amazon FBA MRP`) AS Avg_FBA,
  AVG(`Myntra MRP`) AS Avg_Myntra,
  AVG(`Ajio MRP`) AS Avg_Ajio,
  AVG(`Flipkart MRP`) AS Avg_Flipkart,
  AVG(`Snapdeal MRP`) AS Avg_Snapdeal,
  AVG(`Paytm MRP`) AS Avg_Paytm,
  AVG(`Limeroad MRP`) AS Avg_Limeroad
FROM pricing
GROUP BY Sku
HAVING
  AVG(`Amazon MRP`) IS NOT NULL OR
  AVG(`Amazon FBA MRP`) IS NOT NULL OR
  AVG(`Myntra MRP`) IS NOT NULL OR
  AVG(`Ajio MRP`) IS NOT NULL OR
  AVG(`Flipkart MRP`) IS NOT NULL OR
  AVG(`Snapdeal MRP`) IS NOT NULL OR
  AVG(`Paytm MRP`) IS NOT NULL OR
  AVG(`Limeroad MRP`) IS NOT NULL
ORDER BY Sku
""")

In [84]:
avg_price_per_sku.show(truncate=False)
avg_price_per_sku.write.mode("overwrite").parquet("outputs/avg_price_per_sku")

+------------------------------------+----------+-------+----------+--------+------------+------------+---------+------------+
|Sku                                 |Avg_Amazon|Avg_FBA|Avg_Myntra|Avg_Ajio|Avg_Flipkart|Avg_Snapdeal|Avg_Paytm|Avg_Limeroad|
+------------------------------------+----------+-------+----------+--------+------------+------------+---------+------------+
|3pl Cottom Face Mask Pack of 3      |199.0     |199.0  |199.0     |199.0   |199.0       |199.0       |199.0    |199.0       |
|3pl reusable cotton mask - Pack of 5|299.0     |299.0  |299.0     |299.0   |299.0       |299.0       |299.0    |299.0       |
|COPPER-1276                         |3499.0    |3499.0 |3499.0    |3499.0  |3499.0      |3499.0      |3499.0   |3499.0      |
|FOUR GEMS-2                         |1695.0    |1695.0 |1695.0    |1695.0  |1695.0      |1695.0      |1695.0   |1695.0      |
|GOLD-1351_L                         |4549.0    |4549.0 |4549.0    |4549.0  |4549.0      |4549.0      |4549.0  